In [9]:
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import datetime, timedelta
import pytz

credentials = service_account.Credentials.from_service_account_file(
    '../secrets/google_key.json'
)
client = bigquery.Client(credentials=credentials)

project_id = 'mlbhighlights'
dataset_id = 'dbt_ntaylor'
table_id = '2024_pre'

table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

In [19]:

yesterday = datetime.now() - timedelta(1)
yesterday= yesterday.strftime(f'%Y-%m-%d')

today = datetime.now()
today = today.strftime(f'%Y-%m-%d')


def player_search(player, start_date=yesterday, end_date=today):
    query = f"""
        SELECT * 
        FROM {project_id}.{dataset_id}.{table_id}
        WHERE date >= '{start_date}' and date <= '{end_date}' and player_name like concat('%','{player}','%')
    """
    query_job = client.query(query)
    results = query_job.result()

    highlight_list = []
    for row in results:
        date = row[1]
        name = row[4]
        headline = row[5]
        description = row[6]
        url = row[7]
        highlight = [date,name,headline,description,url]
        highlight_list.append(highlight)
    return highlight_list

def team_search(team, start_date=yesterday, end_date=today):
    query = f"""
        SELECT * 
        FROM {project_id}.{dataset_id}.{table_id}
        WHERE date >= '{start_date}' and date <= '{end_date}' and (away_name = '{team}' or home_name = '{team}')
    """
    query_job = client.query(query)
    results = query_job.result()

    highlight_list = []
    for row in results:
        date = row[1]
        away_name = row[2]
        home_name = row[3]
        player_name = row[4]
        headline = row[5]
        description = row[6]
        url = row[7]
        highlight = [date,away_name,home_name,player_name,headline,description,url]
        highlight_list.append(highlight)
    return highlight_list


In [20]:
list = player_search('Holliday','2024-02-01')
for h in list:
    print(h)

[datetime.date(2024, 3, 10), 'Jackson Holliday', "Jackson Holliday's 1st-inning grand slam", 'Jackson Holliday crushes a grand slam for his first Spring training home run, giving the Orioles a 5-0 lead in the 1st', 'https://mlb-cuts-diamond.mlb.com/FORGE/2024/2024-03/10/c33b7e73-eb559f57-91559294-csvm-diamondx64-asset_1280x720_59_4000K.mp4']
[datetime.date(2024, 2, 29), 'Jackson Holliday', "Check out Jackson Holliday's RBI triple", "Check out a unique angle of Jackson Holliday's game-tying RBI triple in the bottom of the 6th inning against the Pirates #CreatorCuts", 'https://mlb-cuts-diamond.mlb.com/FORGE/2024/2024-02/29/85fb9b6d-463a784d-294a01e4-csvm-diamondx64-asset_1280x720_59_4000K.mp4']
[datetime.date(2024, 2, 29), 'Jackson Holliday', "Radio call of Jackson Holliday's RBI triple", "Listen to the radio call of Jackson Holliday's game-tying RBI triple in the bottom of the 6th inning against the Pirates", 'https://mlb-cuts-diamond.mlb.com/FORGE/2024/2024-02/29/bbd91d72-71d590ca-60ad

In [22]:
list = team_search('Cardinals','2023-01-01')
for h in list:
    print(h)

[datetime.date(2024, 3, 10), 'Cardinals', 'Marlins', 'Edward Cabrera', 'Edward Cabrera against the Cardinals', 'Hits against Cabrera in their last matchup', 'https://darkroom-clips.mlb.com/ccdc885c-1f52-47bd-b364-850dc63b0e99.mp4']
[datetime.date(2024, 3, 10), 'Cardinals', 'Marlins', 'Josh Bell', "Breaking down Josh Bell's home run", "The data behind Josh Bell's home run", 'https://darkroom-clips.mlb.com/c10bbdc0-9db2-476f-ae89-87efacbf1e74.mp4']
[datetime.date(2024, 3, 10), 'Cardinals', 'Marlins', 'Edward Cabrera', 'Edward Cabrera leaves with shoulder tightness', 'Marlins starting pitcher Edward Cabrera leaves the game before the first pitch with right shoulder tightness', 'https://mlb-cuts-diamond.mlb.com/FORGE/2024/2024-03/10/ab1b2b87-22dbd9c0-45a2d472-csvm-diamondx64-asset_1280x720_59_4000K.mp4']
[datetime.date(2024, 3, 10), 'Cardinals', 'Marlins', 'Nolan Arenado', "Nolan Arenado's RBI double to left field", "Nolan Arenado pulls an RBI double down the left-field line to extend the 